In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO

url = 'https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/defense_evasion/host/empire_dllinjection_LoadLibrary_CreateRemoteThread.zip'
zipFileRequest = requests.get(url)
zipFile = ZipFile(BytesIO(zipFileRequest.content))
datasetJSONPath = zipFile.extract(zipFile.namelist()[0])

In [2]:
import pandas as pd
from pandas.io import json

df = json.read_json(path_or_buf=datasetJSONPath, lines=True)

C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
(
df[['@timestamp', 'Hostname', 'SourceImage', 'TargetImage']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 8)
    & (df['StartModule'].str.lower().str.contains('.*kernel32.dll', regex=True))
    & (df['StartFunction'] == 'LoadLibraryA')
]
.head()
)

,@timestamp,Hostname,SourceImage,TargetImage
496,2020-07-22T04:01:04.994Z,WORKSTATION5.mordor.local,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,C:\Windows\System32\notepad.exe


In [4]:
imageLoadDf = (
df[['@timestamp', 'Hostname', 'Image', 'ImageLoaded']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 7)
]
)

fileCreateDf = (
df[['@timestamp', 'Hostname', 'Image', 'TargetFilename']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 11)
]
)

(
pd.merge(imageLoadDf, fileCreateDf,
    left_on = ['ImageLoaded','Hostname'], right_on = ['TargetFilename','Hostname'], how = 'inner')
)

,@timestamp_x,Hostname,Image_x,ImageLoaded,@timestamp_y,Image_y,TargetFilename
